# Import modules

In [ ]:
import pandas as pd
import os
import numpy as np
import datetime

# Setup diretories

In [3]:
# root directory
wdir = os.getcwd()

In [10]:
# References:
# [1] https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a
# [2] https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab
# https://stackoverflow.com/questions/53581278/test-if-notebook-is-running-on-google-colab

try:
    from google.colab import drive
    from google.colab import files
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:

    # mount google drive 
    drive.mount('/content/gdrive/', force_remount=True)

    #change directory
    try:
        os.chdir('/content/gdrive/MyDrive/MCED/TAAC_VC_PROJECT')

    except:
        os.chdir('/content/gdrive/MyDrive/TAAC_VC_PROJECT')

wdir = os.getcwd()

print(wdir)

Mounted at /content/gdrive/
/content/gdrive/.shortcut-targets-by-id/1v5wjtF3FR1_SrbCpPXH3gkApatlnQW4s/TAAC_VC_PROJECT


In [11]:
os.chdir(wdir)

# check for ECMWF directory existence
if not os.path.isdir('ECMWF'):
    os.makedirs('ECMWF')

# check for ECMWF directory existence
if not os.path.isdir('IMAGES'):
    os.makedirs('IMAGES')

ECMWF_dir = wdir + '/ECMWF'
IMAGES_dir = wdir + '/IMAGES'

In [12]:
os.chdir(ECMWF_dir)

os.getcwd()

'/content/gdrive/.shortcut-targets-by-id/1v5wjtF3FR1_SrbCpPXH3gkApatlnQW4s/TAAC_VC_PROJECT/ECMWF'

# Read NetCDF data

In [14]:
#Open the NetCDF

# https://pratiman-91.github.io/2020/08/01/NetCDF-to-GeoTIFF-using-Python.html

import xarray as xr
import rioxarray as rio


#Download the sample from https://www.unidata.ucar.edu/software/netcdf/examples/sresa1b_ncar_ccsm3-example.nc

ncfname = '1985_T2m_tcc_tp.nc'
ncfile = xr.open_dataset(ncfname)

# Inspect the file
ncfile

<xarray.Dataset>
Dimensions:    (longitude: 42, latitude: 30, time: 8760)
Coordinates:
  * longitude  (longitude) float32 -9.07 -9.02 -8.97 -8.92 ... -7.12 -7.07 -7.02
  * latitude   (latitude) float32 41.0 40.95 40.9 40.85 ... 39.65 39.6 39.55
  * time       (time) datetime64[ns] 1985-01-01 ... 1985-12-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    tcc        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-10-30 12:59:39 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [15]:
date= '19850901'

In [16]:
# https://docs.xarray.dev/en/stable/user-guide/indexing.html
date_slice = ncfile.sel(time=slice(date, date))
date_slice

<xarray.Dataset>
Dimensions:    (longitude: 42, latitude: 30, time: 24)
Coordinates:
  * longitude  (longitude) float32 -9.07 -9.02 -8.97 -8.92 ... -7.12 -7.07 -7.02
  * latitude   (latitude) float32 41.0 40.95 40.9 40.85 ... 39.65 39.6 39.55
  * time       (time) datetime64[ns] 1985-09-01 ... 1985-09-01T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    tcc        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-10-30 12:59:39 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [ ]:
date_slice['t2m']

<xarray.DataArray 't2m' (time: 24, latitude: 30, longitude: 42)>
array([[[291.79688, 291.7551 , ..., 292.70184, 292.93692],
        [291.8174 , 291.77493, ..., 292.67633, 292.91144],
        ...,
        [291.56036, 291.46478, ..., 296.02142, 296.1163 ],
        [291.63116, 291.53558, ..., 295.69855, 295.79413]],

       [[291.76004, 291.72534, ..., 292.28476, 292.5262 ],
        [291.77988, 291.74448, ..., 292.27698, 292.51913],
        ...,
        [291.74234, 291.6524 , ..., 295.83237, 295.94565],
        [291.84573, 291.7551 , ..., 295.55408, 295.66666]],

       ...,

       [[291.3026 , 291.29413, ..., 291.3253 , 291.5632 ],
        [291.32245, 291.31393, ..., 291.3373 , 291.57523],
        ...,
        [289.931  , 289.7738 , ..., 295.7828 , 295.8883 ],
        [289.81558, 289.6584 , ..., 295.57532, 295.68082]],

       [[291.3019 , 291.2906 , ..., 291.2233 , 291.48813],
        [291.32245, 291.3104 , ..., 291.2382 , 291.50372],
        ...,
        [290.11935, 289.9636 , ..., 295.35724, 295.4245 ],
        [290.045  , 289.88922, ..., 295.1292 , 295.19577]]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -9.07 -9.02 -8.97 -8.92 ... -7.12 -7.07 -7.02
  * latitude   (latitude) float32 41.0 40.95 40.9 40.85 ... 39.65 39.6 39.55
  * time       (time) datetime64[ns] 1985-09-01 ... 1985-09-01T23:00:00
Attributes:
    units:      K
    long_name:  2 metre temperature

In [ ]:
# https://stackoverflow.com/questions/23943379/swapping-the-dimensions-of-a-numpy-array
# https://numpy.org/doc/stable/reference/generated/numpy.transpose.html

import numpy as np

# temperature
# slices
t2m = date_slice['t2m'].to_numpy()
# change the order of the indices
t2m = np.transpose(t2m, (1, 2, 0))
t2m.shape

(30, 42, 24)

In [ ]:
# total precipitation
tp = date_slice['tp'].to_numpy()
tp = np.transpose(tp, (1, 2, 0))

# total cloud cover
tcc= date_slice['tcc'].to_numpy()
tcc = np.transpose(tcc, (1, 2, 0))


# Save images

In [ ]:
# change directory
os.chdir(IMAGES_dir)

In [ ]:
# https://stackoverflow.com/questions/59307148/python-opencv-how-to-save-a-5-channel-image
np.savez_compressed(date, t2m=t2m, tp=tp, tcc=tcc)

# Load images

In [ ]:
# https://numpy.org/doc/stable/reference/generated/numpy.savez_compressed.html

loaded = np.load(f'{date}.npz')

In [ ]:
t2m_loaded = loaded['t2m']
t2m_loaded

array([[[291.79688, 291.76004, 291.69562, ..., 291.25162, 291.3026 ,
         291.3019 ],
        [291.7551 , 291.72534, 291.66232, ..., 291.24313, 291.29413,
         291.2906 ],
        [291.8167 , 291.8061 , 291.64746, ..., 291.31464, 291.22968,
         291.24667],
        ...,
        [292.46674, 292.04257, 289.91898, ..., 293.77603, 291.08734,
         290.95847],
        [292.70184, 292.28476, 290.14417, ..., 294.02103, 291.3253 ,
         291.2233 ],
        [292.93692, 292.5262 , 290.36932, ..., 294.26675, 291.5632 ,
         291.48813]],

       [[291.8174 , 291.77988, 291.71616, ..., 291.27713, 291.32245,
         291.32245],
        [291.77493, 291.74448, 291.68286, ..., 291.26862, 291.31393,
         291.3104 ],
        [291.83652, 291.8259 , 291.6687 , ..., 291.34015, 291.2495 ,
         291.2665 ],
        ...,
        [292.44125, 292.0348 , 289.89066, ..., 293.83835, 291.0994 ,
         290.97336],
        [292.67633, 292.27698, 290.1151 , ..., 294.08334, 291.3373 ,
   

# Cycles are years and save images to compressed numpy arrays

In [ ]:
# REFERENCES:
# https://stackoverflow.com/questions/23943379/swapping-the-dimensions-of-a-numpy-array
# https://numpy.org/doc/stable/reference/generated/numpy.transpose.html

import numpy as np
from datetime import datetime, timedelta

def save_npz(ncfile, date):
    # https://docs.xarray.dev/en/stable/user-guide/indexing.html
    date_slice = ncfile.sel(time=slice(date, date))


    # https://stackoverflow.com/questions/23943379/swapping-the-dimensions-of-a-numpy-array
    # https://numpy.org/doc/stable/reference/generated/numpy.transpose.html

    # temperature
    t2m = date_slice['t2m'].to_numpy()
    t2m = np.transpose(t2m, (1, 2, 0))

    # total precipitation
    tp = date_slice['tp'].to_numpy()
    tp = np.transpose(tp, (1, 2, 0))

    # total cloud cover

    tcc= date_slice['tcc'].to_numpy()
    tcc = np.transpose(tcc, (1, 2, 0))

    # display(date)

    # https://stackoverflow.com/questions/59307148/python-opencv-how-to-save-a-5-channel-image
    np.savez_compressed(date, t2m=t2m, tp=tp, tcc=tcc)


# Change diretory

In [ ]:
os.chdir(ECMWF_dir)

os.getcwd()

'h:\\My Drive\\MCED\\TAAC_VC_PROJECT\\ECMWF'

In [ ]:
# list of years to process

year_start = 1985
year_end = 1995

In [ ]:
# https://stackoverflow.com/questions/61810757/find-total-number-of-days-in-a-year-pandas
import datetime
import calendar

# function to compute the number of days in a year
def days_in_year(year=datetime.datetime.now().year):
    return 365 + calendar.isleap(year)


In [ ]:
from datetime import datetime, timedelta
from tqdm import tqdm

for year in range(year_start, year_end + 1):

    ndays = days_in_year(year)

    day_start = datetime(year, 1, 1, 0, 0)

    os.chdir(ECMWF_dir)
    ncfname = str(year) + '_T2m_tcc_tp.nc'
    ncfile = xr.open_dataset(ncfname)

    print(f'Processing file {ncfname}')

    for d in tqdm(range(ndays)):
        # Compute current date
        date = day_start + timedelta(days = d)
        date = date.strftime('%Y%m%d')
        try:
            os.chdir(IMAGES_dir)
            save_npz(ncfile, date)
        except:
            continue


Processing file 1985_T2m_tcc_tp.nc


100%|██████████| 365/365 [00:28<00:00, 12.60it/s]


Processing file 1986_T2m_tcc_tp.nc


100%|██████████| 365/365 [00:33<00:00, 11.00it/s]


Processing file 1987_T2m_tcc_tp.nc


...

Processing file 1994_T2m_tcc_tp.nc


100%|██████████| 365/365 [01:22<00:00,  4.43it/s]


Processing file 1995_T2m_tcc_tp.nc


100%|██████████| 365/365 [01:37<00:00,  3.74it/s]


FileNotFoundError: [Errno 2] No such file or directory: b'h:\\My Drive\\MCED\\TAAC_VC_PROJECT\\ECMWF\\1996_T2m_tcc_tp.nc'